## Implicit Library

In [43]:
import implicit
import h5py
import pandas as pd
import numpy as np
import random
import statistics
from implicit.evaluation import train_test_split
from sklearn.metrics import ndcg_score
from numpy.random import permutation
from implicit.nearest_neighbours import (
    BM25Recommender,
    CosineRecommender,
    TFIDFRecommender,
    bm25_weight,
)
import scipy
from scipy.sparse import csr_matrix
from implicit.lmf import LogisticMatrixFactorization

## Preprocessing

In [2]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [3]:
data = df

In [4]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [5]:
data = df

In [6]:
matrix = pd.read_csv("data/origin_binary_matrix.csv")

In [7]:
matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Normalizing the rows

In [8]:
data_skills  = matrix
data_skills 

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In the next step we will compute the item-item relationships of our skills. Our final goal here is to construct a new item by item matrix containing the weights (relationships) between each of our skills where a perfect correlation equals 1 and no correlation at all equals 0.
<br>
In order to do so, we will first normalize the user vectors. The idea behind this approach is, that a user with many skills contributes less to any individual skill. For example if a user, that rules only 3 skills, rules a skill X it is more valueable than if a user, that rules 20 skills, that particular skill.


* First we caclulat the magnitude for every user

In [9]:
# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_skills).sum(axis=1))

In [10]:
magnitude

0       8.774964
1       8.602325
2       5.567764
3       5.567764
4       1.000000
         ...    
399     2.828427
400     4.358899
401     3.000000
402     2.645751
403    12.206556
Length: 404, dtype: float64

* Now we use a users magnitude to normalize the ratings of this corresponding user

In [11]:
data_skills_row_norm=  data_skills.divide(magnitude, axis='index')

In [12]:
data_skills_row_norm

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.116248,0.000000,0.000000,0.116248,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.000000,0.179605,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.179605,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
400,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
401,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
402,0.0,0.377964,0.377964,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


### Create sparse Matrix representation for the implicit lib

In [13]:
data_skills_row_norm_csr = scipy.sparse.csr_matrix(data_skills_row_norm.values)


In [14]:
data_skills_row_norm_csr?

Type:        csr_matrix
String form:
(0, 22)	0.11396057645963795
           (0, 30)	0.11396057645963795
           (0, 35)	0.11396057645963795
           (0, 4 <...>  (403, 713)	0.08192319205190406
           (403, 719)	0.08192319205190406
           (403, 730)	0.08192319205190406
File:        c:\users\maximus\envs\ds_ap\lib\site-packages\scipy\sparse\_csr.py
Docstring:  
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standar

In [123]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors = 1,random_state=1,regularization= 0.005) 

In [124]:
# train the model on a sparse matrix of user/item/confidence weights
model.fit(data_skills_row_norm_csr)

100%|██████████| 15/15 [00:00<00:00, 43.00it/s]


In [116]:
userid= 13
dict_user_skills = {}
for value, name in zip(data_skills_row_norm.iloc[userid,:], data_skills_row_norm.columns):
    if value > 0:
        dict_user_skills[name] = value
    else:
        next

dict_user_skills

{'Android': 0.23570226039551587,
 'Apache': 0.23570226039551587,
 'GIT': 0.23570226039551587,
 'JavaScript': 0.23570226039551587,
 'Joomla': 0.23570226039551587,
 'Microsoft SQL Server': 0.23570226039551587,
 'MySQL': 0.23570226039551587,
 'Oracle': 0.23570226039551587,
 'PHP': 0.23570226039551587,
 'PhpStorm': 0.23570226039551587,
 'Pimcore': 0.23570226039551587,
 'SUSE Linux': 0.23570226039551587,
 'Subversion': 0.23570226039551587,
 'Symfony': 0.23570226039551587,
 'Ubuntu': 0.23570226039551587,
 'Windows 10': 0.23570226039551587,
 'Windows 7': 0.23570226039551587,
 'Windows NT/2000/XP': 0.23570226039551587}

### Create a df that holds the recommendations and the scores 
* Die Scores sind in der implicip library equivalent zu der confidence. Die Confidence gibt an, wie häufig ein item x konsumiert wurde. Allerdings soll die Intensität, mit der dieser Effekt steigt, sukkzessive abnehmen, um den Einfluss eines Superfans (jmd der überdurchschnittkich viel von Item x konsumiert) zu schwächen. Hier sind parallelen zu den Motiven der Row-Normalisation zu erkennen. <br>
Ich habe im ersten Lauf die Werte meiner row-normalisation als confidence werte verwendet. <br>
    * Sollte nach angepasst werden. Hierzu: https://en.wikipedia.org/wiki/Okapi_BM25

Maybe it makes sense <br>
"
Confidence can be defined as the worth or the value we give to the interaction. For User A buying(a transaction event) item X we increase the interaction weight, while User A viewing item Z has lesser weight than the ‘interaction of buying’. https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b " 

In [146]:
skill_ids, scores  = model.recommend(userid, data_skills_row_norm_csr[userid], N = 20 , filter_already_liked_items=False)
recommendations_df = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, data_skills_row_norm_csr[userid].indices)})
recommendations_df

,skill,score,already_liked
0,Englisch,0.109891,False
1,MySQL,0.108704,True
2,HTML,0.081065,False
3,JavaScript,0.072065,True
4,SQL,0.071913,False
5,GIT,0.069570,True
6,SCRUM,0.061550,False
7,Test Driven Development (TDD),0.056983,False
8,Java,0.056517,False
9,JIRA,0.056304,False


In [118]:
skill_and_categories = df[["skill","category"]].drop_duplicates()

KeyError: "['category'] not in index"

In [125]:
recommendations_df.merge(skill_and_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,already_liked,category
0,MySQL,0.440420,True,Datenbank
1,PHP,0.422259,True,Webentwicklung
2,Windows NT/2000/XP,0.380840,True,Betriebssystem
3,Apache,0.380116,True,Applikationsserver
4,TYPO3 Generalist,0.216580,False,TYPO3
5,PhpStorm,0.204278,True,Umgebungen
6,Windows 7,0.195201,True,Betriebssystem
7,Java,0.185855,False,Programmiersprachen / Scriptsprachen
8,Englisch,0.185376,False,Sprachen
9,C/C++,0.167171,False,Programmiersprachen / Scriptsprachen


In [120]:
# Id and skills
dict_skills_id = {}
for c,skill in enumerate(data_skills_row_norm.columns):
        dict_skills_id[c] = skill

dict_skills_id

{0: '.NET Compact Framework',
 1: '.NET Core',
 2: '.NET Framework',
 3: '3D-Modellierung',
 4: 'ABAP',
 5: 'ADO.NET',
 6: 'AIX',
 7: 'ARIS',
 8: 'ARIS ITArchitect',
 9: 'AS400',
 10: 'ASP Generalist',
 11: 'ASP.NET',
 12: 'ASP.NET MVC-Framework',
 13: 'ASP.NET WebAPI',
 14: 'AWS',
 15: 'AWS Lambda',
 16: 'Abstract',
 17: 'Access',
 18: 'Accessibility / WCAG',
 19: 'Active Directory',
 20: 'ActiveX',
 21: 'Adobe CC',
 22: 'Adobe Flash',
 23: 'Adobe Illustrator',
 24: 'Adobe InDesign',
 25: 'Adobe Indesign',
 26: 'Adobe Photoshop',
 27: 'Adobe Premiere',
 28: 'Adobe XD',
 29: 'After Effects',
 30: 'Agile Methoden',
 31: 'Ajax',
 32: 'Alexa-Skills',
 33: 'Alpine',
 34: 'Analytics',
 35: 'Android',
 36: 'Android Studio',
 37: 'Anforderungsanalyse',
 38: 'Anforderungsmanagement',
 39: 'Angular (2 und höher)',
 40: 'Angular Generalist',
 41: 'Angular Material',
 42: 'Angular Theming',
 43: 'AngularJS',
 44: 'Animations (transition, @keyframes)',
 45: 'Ansible',
 46: 'Ant',
 47: 'Apache',
 4

In [22]:
keys = [k for k, v in dict_skills_id.items() if v == 'Python']
print(keys)

[458]


### Display simular items

In [23]:
# find related item
skill_id, sim = model.similar_items(0, N=10)
# display the results using pandas for nicer formatting
simularity_df  = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_id], "score": sim})
simularity_df.merge(skill_and_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,category
0,.NET Compact Framework,1.000000,.NET Frameworks und Tools
1,ADO.NET,0.674586,.NET Frameworks und Tools
2,DevExpress,0.635214,.NET Frameworks und Tools
3,Silverlight,0.632719,.NET Frameworks und Tools
4,Microsoft Enterprise Library,0.632117,.NET Frameworks und Tools
5,ASP Generalist,0.612088,.NET Frameworks und Tools
6,Windows Communication Foundation (WCF),0.599716,.NET Frameworks und Tools
7,Mono,0.578717,.NET Frameworks und Tools
8,Windows Forms,0.566209,.NET Frameworks und Tools
9,Exact,0.548891,ERP


# Add a Evaluation metric to the library

### APK
http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

In [24]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [121]:
###
# Recommendations for user_id = 13
###
recommendations_list = list(recommendations_df.skill)
known_skills_list = list(dict_user_skills.keys())
len(known_skills_list)

18

In [122]:
apk(known_skills_list,recommendations_list,k=20)

0.4013227513227513

## Train-Test split

In [135]:
# train the model on a sparse matrix of user/item/confidence weights
train, test = train_test_split(data_skills_row_norm_csr, train_percentage =  0.8, random_state = 1)
model = implicit.als.AlternatingLeastSquares(factors = 1, random_state=1, regularization= 0.05)
model.fit(train)

100%|██████████| 15/15 [00:00<00:00, 21.35it/s]


In [139]:
test[0]

<1x735 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [143]:
matrix.iloc[0].sum()

77

In [161]:
user_id  = 3
skill_ids, scores = model.recommend(0, data_skills_row_norm_csr[user_id], N = 4 , filter_already_liked_items=False)
recommendations_df = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, data_skills_row_norm_csr[userid].indices)})
recommendations_df

,skill,score,already_liked
0,Englisch,0.436307,False
1,MySQL,0.431594,True
2,HTML,0.321858,False
3,JavaScript,0.286125,True


In [151]:
most_common_skills = matrix.sum(axis=0).nlargest(10)	
for i in most_common_skills.index:
    print(i)

Englisch
MySQL
SCRUM
GIT
HTML
JavaScript
JIRA
CSS
SQL
Java


In [28]:
from implicit.evaluation import precision_at_k,mean_average_precision_at_k, ndcg_at_k

In [142]:
results = precision_at_k(model, train_user_items=train, test_user_items=test, K=4)
results

100%|██████████| 378/378 [00:00<00:00, 2075.70it/s]


0.45878136200716846

In [30]:
results = mean_average_precision_at_k(model, train_user_items=train, test_user_items=test, K=5)
results

100%|██████████| 378/378 [00:00<00:00, 2592.76it/s]


0.16690255731922415

In [31]:
mean_average_precision_at_k?

Docstring:
mean_average_precision_at_k(model, train_user_items, test_user_items, int K=10, show_progress=True, int num_threads=1)
Calculates MAP@K for a given trained model

   Parameters
   ----------
   model : RecommenderBase
       The fitted recommendation model to test
   train_user_items : csr_matrix
       Sparse matrix of user by item that contains elements that were used in training the model
   test_user_items : csr_matrix
       Sparse matrix of user by item that contains withheld elements to test on
   K : int
       Number of items to test on
   show_progress : bool, optional
       Whether to show a progress bar
   num_threads : int, optional
       The number of threads to use for testing. Specifying 0 means to default
       to the number of cores on the machine. Note: aside from the ALS and BPR
       models, setting this to more than 1 will likely hurt performance rather than
       help.

   Returns
   -------
   float
       the calculated MAP@k
   
Type:      buil

## Crossvalidation

In [32]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [33]:
df = df.drop(columns=["category"])

In [34]:
data = df
data.head(5)

,person,skill
0,12,Windows NT/2000/XP
1,12,MAC OS X
2,12,Windows 7
3,12,Windows 8
4,12,iOS


In [36]:
NUM_TRIALS = 5
NUM_OUTER_SPLITS = 5

In [37]:
from sklearn.model_selection import cross_validate,KFold
from sklearn.metrics import make_scorer
from random import shuffle


In [38]:
folds = 4

results = pd.DataFrame()
def cross_validation_implicit(data):
    results_over_all = pd.DataFrame(columns=["factors","alpha","regularization","p_at_k_mean","map_at_k_mean","ndcg_at_k_mean"])
    factor_list = [1,2,3,4]
    alpha_list = [0.5,1.0, 1.5]
    regularization_list = [0.005,0.01,0.02]


    for f in factor_list:
        for a in alpha_list:
            for r in regularization_list:
                avg_p_at_k_list = []
                avg_map_at_k_list = []
                avg_ndcg_at_k_list = []

                #Note: Shuffle is set on false, because it leads to ambiguous results regarding the map_at_k and the ndcg_at_k. The 
                # However p_at_k is not affected by the shuffle and stays consistent.
                skf = KFold(n_splits= folds , shuffle=False) 

                for r_seed in range(0,5):
                    # Lists to store the results for each random_seed
                    p_at_k_list = []
                    map_at_k_list = []
                    ndcg_at_k_list = []
                    model = implicit.als.AlternatingLeastSquares(factors = f, random_state= r_seed, alpha= a, regularization= r)

                    ###########################
                    # Split the data
                    ###########################
                    for train_index, test_index in skf.split(data):
                        X_train = data[train_index]
                        X_test = data[test_index]
                        
                        ###########################
                        # Shuffle manually
                        ###########################

                        index = np.arange(np.shape(X_train)[0])
                        np.random.RandomState(seed = 1).shuffle(index)
                        
                        X_train = X_train[index, :] 
                        model.fit(X_train)

                        #############
                        # Fifth Step: We predict on the scaled test data
                        #############
                        p_at_k = precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        map_at_k  = mean_average_precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        var_ndcg_at_k = ndcg_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)

                        # results to list
                        p_at_k_list.append(p_at_k)
                        map_at_k_list.append(map_at_k) 
                        ndcg_at_k_list.append(var_ndcg_at_k) #store as df to create the avg confusion matrix later


                    # store the results for each random_seed
                    # After each iteration in the random_seed loop the lists (acc_list, bacc_list and confusion_m_list) are cleaned. Therefore we can do the following computations to store the average results for each random_seed:
                    # Store the averages per random_seed in list. 
                    avg_p_at_k_list.append(statistics.mean(p_at_k_list))
                    avg_map_at_k_list.append(statistics.mean(map_at_k_list))
                    avg_ndcg_at_k_list.append(statistics.mean(ndcg_at_k_list))

                
                row = {
                    "factors": f,
                    "alpha": a,
                    "regularization": r,
                    "p_at_k_mean": statistics.mean(avg_p_at_k_list),  
                    "map_at_k_mean":statistics.mean(avg_map_at_k_list), 
                    "ndcg_at_k_mean":statistics.mean(avg_ndcg_at_k_list),
                    }
                results_per_run = pd.DataFrame([row])
                results_over_all = pd.concat([results_over_all, results_per_run], ignore_index=True)
    return results_over_all
                
              #  return avg_p_at_k_list,avg_map_at_k_list,avg_ndcg_at_k_list


In [39]:
results_of_als_grid = cross_validation_implicit(data_skills_row_norm_csr)
results_of_als_grid

100%|██████████| 101/101 [00:00<00:00, 2740.58it/s]


,factors,alpha,regularization,p_at_k_mean,map_at_k_mean,ndcg_at_k_mean
0,1,0.5,0.005,0.570055,0.480130,0.552944
1,1,0.5,0.010,0.568256,0.478310,0.551325
2,1,0.5,0.020,0.567724,0.477992,0.550915
3,1,1.0,0.005,0.567751,0.478264,0.551782
4,1,1.0,0.010,0.566586,0.477153,0.550738
5,1,1.0,0.020,0.567751,0.478264,0.551782
6,1,1.5,0.005,0.571134,0.480731,0.553795
7,1,1.5,0.010,0.570820,0.480434,0.553600
8,1,1.5,0.020,0.570611,0.480236,0.553470
9,2,0.5,0.005,0.524033,0.432543,0.507300


In [ ]:
results_of_als_grid.to_csv("results_of_als_grid.csv", index=False)

In [ ]:
folds = 4

results = pd.DataFrame()
def cross_validation_implicit(data):
    results_over_all = pd.DataFrame(columns=["factors","alpha","regularization","p_at_k_mean","map_at_k_mean","ndcg_at_k_mean"])
    factor_list = [1,2,3,4]
    alpha_list = [0.5,1.0, 1.5]
    regularization_list = [0.005,0.01,0.02]


    for f in factor_list:
        for a in alpha_list:
            for r in regularization_list:
                avg_p_at_k_list = []
                avg_map_at_k_list = []
                avg_ndcg_at_k_list = []

                #Note: Shuffle is set on false, because it leads to ambiguous results regarding the map_at_k and the ndcg_at_k. The 
                # However p_at_k is not affected by the shuffle and stays consistent.
                skf = KFold(n_splits= folds , shuffle=False) 

                for r_seed in range(0,5):
                    # Lists to store the results for each random_seed
                    p_at_k_list = []
                    map_at_k_list = []
                    ndcg_at_k_list = []
                    model = implicit.als.AlternatingLeastSquares(factors = f, random_state= r_seed, alpha= a, regularization= r)

                    ###########################
                    # Split the data
                    ###########################
                    for train_index, test_index in skf.split(data):
                        X_train = data[train_index]
                        X_test = data[test_index]
                        
                        ###########################
                        # Shuffle manually
                        ###########################

                        index = np.arange(np.shape(X_train)[0])
                        np.random.RandomState(seed = 1).shuffle(index)
                        
                        X_train = X_train[index, :] 
                        model.fit(X_train)

                        #############
                        # Fifth Step: We predict on the scaled test data
                        #############
                        p_at_k = precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        map_at_k  = mean_average_precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        var_ndcg_at_k = ndcg_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)

                        # results to list
                        p_at_k_list.append(p_at_k)
                        map_at_k_list.append(map_at_k) 
                        ndcg_at_k_list.append(var_ndcg_at_k) #store as df to create the avg confusion matrix later


                    # store the results for each random_seed
                    # After each iteration in the random_seed loop the lists (acc_list, bacc_list and confusion_m_list) are cleaned. Therefore we can do the following computations to store the average results for each random_seed:
                    # Store the averages per random_seed in list. 
                    avg_p_at_k_list.append(statistics.mean(p_at_k_list))
                    avg_map_at_k_list.append(statistics.mean(map_at_k_list))
                    avg_ndcg_at_k_list.append(statistics.mean(ndcg_at_k_list))

                
                row = {
                    "factors": f,
                    "alpha": a,
                    "regularization": r,
                    "p_at_k_mean": statistics.mean(avg_p_at_k_list),  
                    "map_at_k_mean":statistics.mean(avg_map_at_k_list), 
                    "ndcg_at_k_mean":statistics.mean(avg_ndcg_at_k_list),
                    }
                results_per_run = pd.DataFrame([row])
                results_over_all = pd.concat([results_over_all, results_per_run], ignore_index=True)
    return results_over_all
                

In [ ]:
def add_result(results,name, p_at_k,map_at_k,ndcg_at_k):
    """This function appends a name, an average accuravy, an average balanced accuracy and a average confusion matrix as rows to a dataframe.

    Args:
        results (dataframe): A dataframe to append the other arguments as rows
        name (string): Identifier
        acc (list of floats): computes the average accuracy
        baccs (list of floats): computes the average balanced accuracy
        confusion_m (list of arrays): computes the average Confusion Matrix
    """
    row = {
        "name":name,
        "p_at_k_mean": statistics.mean(p_at_k),  
        "map_at_k_mean":statistics.mean(map_at_k), 
        "ndcg_at_k_mean":statistics.mean(ndcg_at_k),
        }
    return results.append(row,ignore_index=True)

results = pd.DataFrame()

In [ ]:
def add_result2(results,f,a,r, p_at_k,map_at_k,ndcg_at_k):
    """This function appends a name, an average accuravy, an average balanced accuracy and a average confusion matrix as rows to a dataframe.

    Args:
        results (dataframe): A dataframe to append the other arguments as rows
        name (string): Identifier
        acc (list of floats): computes the average accuracy
        baccs (list of floats): computes the average balanced accuracy
        confusion_m (list of arrays): computes the average Confusion Matrix
    """
    row = {
        "factors":f,
        "alpha":a,
        "regularization":r,
        "p_at_k_mean": statistics.mean(p_at_k),  
        "map_at_k_mean":statistics.mean(map_at_k), 
        "ndcg_at_k_mean":statistics.mean(ndcg_at_k),
        }
    return results.append(row,ignore_index=True)

results = pd.DataFrame()

In [ ]:
p_k, map_k, ndcg_k = cross_validation_implicit(data=data_skills_row_norm_csr)
results = add_result(results, "ALS",p_k,map_k, ndcg_k)
results

100%|██████████| 101/101 [00:00<00:00, 3156.13it/s]
C:\Users\Maximus\AppData\Local\Temp\ipykernel_19128\3541173182.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return results.append(row,ignore_index=True)


,name,p_at_k_mean,map_at_k_mean,ndcg_at_k_mean
0,ALS,0.418204,0.309401,0.40115
1,ALS,0.418204,0.309401,0.40115
2,ALS,0.418204,0.309401,0.40115
3,ALS,0.418204,0.309401,0.40115


In [ ]:
from sklearn.model_selection import GridSearchCV


In [41]:

folds = 4

results = pd.DataFrame()
def cross_validation_implicit_bpr(data):
    results_over_all = pd.DataFrame(columns=["factors","regularization","p_at_k_mean","map_at_k_mean","ndcg_at_k_mean"])
    factor_list = [1,5,10,20,50]
    regularization_list = [0.005,0.01,0.02]


    for f in factor_list:
        for r in regularization_list:
            avg_p_at_k_list = []
            avg_map_at_k_list = []
            avg_ndcg_at_k_list = []

            #Note: Shuffle is set on false, because it leads to ambiguous results regarding the map_at_k and the ndcg_at_k. The 
            # However p_at_k is not affected by the shuffle and stays consistent.
            skf = KFold(n_splits= folds , shuffle=False) 

            for r_seed in range(0,5):
                # Lists to store the results for each random_seed
                p_at_k_list = []
                map_at_k_list = []
                ndcg_at_k_list = []
                model = implicit.bpr.BayesianPersonalizedRanking(factors = f, random_state= r_seed, regularization= r)

                ###########################
                # Split the data
                ###########################
                for train_index, test_index in skf.split(data):
                    X_train = data[train_index]
                    X_test = data[test_index]
                    
                    ###########################
                    # Shuffle manually
                    ###########################

                    index = np.arange(np.shape(X_train)[0])
                    np.random.RandomState(seed = 1).shuffle(index)
                    
                    X_train = X_train[index, :] 
                    model.fit(X_train)

                    #############
                    # Fifth Step: We predict on the scaled test data
                    #############
                    p_at_k = precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                    map_at_k  = mean_average_precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                    var_ndcg_at_k = ndcg_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)

                    # results to list
                    p_at_k_list.append(p_at_k)
                    map_at_k_list.append(map_at_k) 
                    ndcg_at_k_list.append(var_ndcg_at_k) #store as df to create the avg confusion matrix later


                # store the results for each random_seed
                # After each iteration in the random_seed loop the lists (acc_list, bacc_list and confusion_m_list) are cleaned. Therefore we can do the following computations to store the average results for each random_seed:
                # Store the averages per random_seed in list. 
                avg_p_at_k_list.append(statistics.mean(p_at_k_list))
                avg_map_at_k_list.append(statistics.mean(map_at_k_list))
                avg_ndcg_at_k_list.append(statistics.mean(ndcg_at_k_list))

            
            row = {
                "factors": f,
                "regularization": r,
                "p_at_k_mean": statistics.mean(avg_p_at_k_list),  
                "map_at_k_mean":statistics.mean(avg_map_at_k_list), 
                "ndcg_at_k_mean":statistics.mean(avg_ndcg_at_k_list),
                }
            results_per_run = pd.DataFrame([row])
            results_over_all = pd.concat([results_over_all, results_per_run], ignore_index=True)
    return results_over_all
            
              #  return avg_p_at_k_list,avg_map_at_k_list,avg_ndcg_at_k_list


In [42]:
results_of_bpr_grid = cross_validation_implicit_bpr(data_skills_row_norm_csr)
results_of_bpr_grid

100%|██████████| 101/101 [00:00<00:00, 3565.26it/s]


,factors,regularization,p_at_k_mean,map_at_k_mean,ndcg_at_k_mean
0,1,0.005,0.424007,0.344604,0.420083
1,1,0.010,0.423791,0.344165,0.420290
2,1,0.020,0.424949,0.344927,0.421517
3,5,0.005,0.254302,0.186533,0.253958
4,5,0.010,0.253880,0.185932,0.253187
5,5,0.020,0.260535,0.192172,0.259911
6,10,0.005,0.225367,0.159818,0.225753
7,10,0.010,0.223653,0.160471,0.225264
8,10,0.020,0.233774,0.168269,0.234268
9,20,0.005,0.208391,0.144956,0.206501


In [167]:

folds = 4

results = pd.DataFrame()
def cross_validation_implicit_lmf(data):
    results_over_all = pd.DataFrame(columns=["factors","epochs","regularization","p_at_k_mean","map_at_k_mean","ndcg_at_k_mean"])
    factor_list = [5,10,20,50,70]
    iteration_list = [20,30, 50,70]
    regularization_list = [0.005,0.01,0.02]


    for f in factor_list:
        for i in iteration_list:
            for r in regularization_list:
                avg_p_at_k_list = []
                avg_map_at_k_list = []
                avg_ndcg_at_k_list = []

                #Note: Shuffle is set on false, because it leads to ambiguous results regarding the map_at_k and the ndcg_at_k. The 
                # However p_at_k is not affected by the shuffle and stays consistent.
                skf = KFold(n_splits= folds , shuffle=False) 

                for r_seed in range(0,5):
                    # Lists to store the results for each random_seed
                    p_at_k_list = []
                    map_at_k_list = []
                    ndcg_at_k_list = []
                    model = implicit.lmf.LogisticMatrixFactorization(factors = f, random_state= r_seed, iterations = i, regularization= r)

                    ###########################
                    # Split the data
                    ###########################
                    for train_index, test_index in skf.split(data):
                        X_train = data[train_index]
                        X_test = data[test_index]
                        
                        ###########################
                        # Shuffle manually
                        ###########################

                        index = np.arange(np.shape(X_train)[0])
                        np.random.RandomState(seed = 1).shuffle(index)
                        
                        X_train = X_train[index, :] 
                        model.fit(X_train)

                        #############
                        # Fifth Step: We predict on the scaled test data
                        #############
                        p_at_k = precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        map_at_k  = mean_average_precision_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)
                        var_ndcg_at_k = ndcg_at_k(model, train_user_items=X_train, test_user_items=X_test, K=5)

                        # results to list
                        p_at_k_list.append(p_at_k)
                        map_at_k_list.append(map_at_k) 
                        ndcg_at_k_list.append(var_ndcg_at_k) #store as df to create the avg confusion matrix later


                    # store the results for each random_seed
                    # After each iteration in the random_seed loop the lists (acc_list, bacc_list and confusion_m_list) are cleaned. Therefore we can do the following computations to store the average results for each random_seed:
                    # Store the averages per random_seed in list. 
                    avg_p_at_k_list.append(statistics.mean(p_at_k_list))
                    avg_map_at_k_list.append(statistics.mean(map_at_k_list))
                    avg_ndcg_at_k_list.append(statistics.mean(ndcg_at_k_list))

                
                row = {
                    "factors": f,
                    "epochs": i,
                    "regularization": r,
                    "p_at_k_mean": statistics.mean(avg_p_at_k_list),  
                    "map_at_k_mean":statistics.mean(avg_map_at_k_list), 
                    "ndcg_at_k_mean":statistics.mean(avg_ndcg_at_k_list),
                    }
                results_per_run = pd.DataFrame([row])
                results_over_all = pd.concat([results_over_all, results_per_run], ignore_index=True)
    return results_over_all

In [168]:
results_of_bpr_lmf = cross_validation_implicit_lmf(data_skills_row_norm_csr)
results_of_bpr_lmf

100%|██████████| 101/101 [00:00<00:00, 1836.39it/s]


,factors,epochs,regularization,p_at_k_mean,map_at_k_mean,ndcg_at_k_mean
0,5,20,0.005,0.211345,0.132021,0.198403
1,5,20,0.010,0.226556,0.145399,0.214212
2,5,20,0.020,0.231750,0.147755,0.218211
3,5,30,0.005,0.196010,0.125619,0.185240
4,5,30,0.010,0.198243,0.126151,0.186596
5,5,30,0.020,0.214959,0.136981,0.202376
6,5,50,0.005,0.161072,0.099616,0.148457
7,5,50,0.010,0.171202,0.110067,0.161700
8,5,50,0.020,0.189409,0.121717,0.177791
9,5,70,0.005,0.142384,0.090614,0.134679


In [169]:
results_of_bpr_lmf.dtypes

factors            object
epochs             object
regularization    float64
p_at_k_mean       float64
map_at_k_mean     float64
ndcg_at_k_mean    float64
dtype: object

In [174]:
results_of_bpr_lmf.describe()

,regularization,p_at_k_mean,map_at_k_mean,ndcg_at_k_mean
count,60.000000,60.000000,60.000000,60.000000
mean,0.011667,0.216070,0.140182,0.201202
std,0.006289,0.049113,0.037761,0.046637
min,0.005000,0.129559,0.080903,0.117947
25%,0.005000,0.173136,0.112629,0.163186
50%,0.010000,0.213593,0.134124,0.199056
75%,0.020000,0.250388,0.160673,0.233339
max,0.020000,0.294134,0.205017,0.275885


In [170]:
results_of_bpr_lmf.p_at_k_mean.idxmax()
results_of_bpr_lmf.iloc[results_of_bpr_lmf.p_at_k_mean.idxmax()]

factors                 70
epochs                  70
regularization        0.02
p_at_k_mean       0.294134
map_at_k_mean     0.205017
ndcg_at_k_mean    0.274624
Name: 59, dtype: object

In [171]:
results_of_bpr_lmf.iloc[results_of_bpr_lmf.map_at_k_mean.idxmax()]

factors                 70
epochs                  70
regularization        0.02
p_at_k_mean       0.294134
map_at_k_mean     0.205017
ndcg_at_k_mean    0.274624
Name: 59, dtype: object

In [173]:
results_of_bpr_lmf.iloc[results_of_bpr_lmf.ndcg_at_k_mean.idxmax()]

factors                 50
epochs                  50
regularization        0.02
p_at_k_mean       0.292184
map_at_k_mean     0.203665
ndcg_at_k_mean    0.275885
Name: 44, dtype: object

In [203]:
# train the model on a sparse matrix of user/item/confidence weights
train, test = train_test_split(data_skills_row_norm_csr, train_percentage =  0.8, random_state = 1)
model = implicit.lmf.LogisticMatrixFactorization(factors = 70, random_state=1,iterations = 50, regularization= 0.02)
model.fit(train)


100%|██████████| 50/50 [00:04<00:00, 10.86it/s]


In [183]:
test?

Type:        csr_matrix
String form:
(0, 165)	0.11396057645963795
           (0, 242)	0.11396057645963795
           (0, 243)	0.11396057645963795
           (0 <...>  (403, 629)	0.08192319205190406
           (403, 643)	0.08192319205190406
           (403, 652)	0.08192319205190406
File:        c:\users\maximus\envs\ds_ap\lib\site-packages\scipy\sparse\_csr.py
Docstring:  
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standar

In [198]:
matrix.iloc[0].sum()


77

In [210]:
user_id  = 0
skill_ids, scores = model.recommend(user_id, data_skills_row_norm_csr[user_id], N = 10 , filter_already_liked_items=False)
recommendations_df = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, data_skills_row_norm_csr[userid].indices)})
recommendations_df

,skill,score,already_liked
0,OpenOffice / LibreOffice,0.580505,False
1,Subversion,0.305606,True
2,PowerShell,0.297782,False
3,JSON,0.220934,False
4,Windows NT/2000/XP,0.176941,True
5,Spanisch,0.072854,False
6,Wasserfallmodel,-0.120552,False
7,Android,-0.179944,True
8,Windows 10,-0.215220,True
9,JavaScript,-0.330638,True


In [211]:
recommendations_df.merge(skill_and_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,already_liked,category
0,OpenOffice / LibreOffice,0.580505,False,Produkterfahrung
1,Subversion,0.305606,True,"CI/CD, Build- und Versionskontrollsysteme"
2,PowerShell,0.297782,False,Technik/Tools
3,JSON,0.220934,False,Webentwicklung
4,Windows NT/2000/XP,0.176941,True,Betriebssystem
5,Spanisch,0.072854,False,Sprachen
6,Wasserfallmodel,-0.120552,False,Projektmanagement / Vorgehensmodelle
7,Android,-0.179944,True,Betriebssystem
8,Windows 10,-0.215220,True,Betriebssystem
9,JavaScript,-0.330638,True,Programmiersprachen / Scriptsprachen


In [206]:
dict_user_skills = {}
for value, name in zip(data_skills_row_norm.iloc[userid,:], data_skills_row_norm.columns):
    if value > 0:
        dict_user_skills[name] = value
    else:
        next

dict_user_skills

{'Android': 0.23570226039551587,
 'Apache': 0.23570226039551587,
 'GIT': 0.23570226039551587,
 'JavaScript': 0.23570226039551587,
 'Joomla': 0.23570226039551587,
 'Microsoft SQL Server': 0.23570226039551587,
 'MySQL': 0.23570226039551587,
 'Oracle': 0.23570226039551587,
 'PHP': 0.23570226039551587,
 'PhpStorm': 0.23570226039551587,
 'Pimcore': 0.23570226039551587,
 'SUSE Linux': 0.23570226039551587,
 'Subversion': 0.23570226039551587,
 'Symfony': 0.23570226039551587,
 'Ubuntu': 0.23570226039551587,
 'Windows 10': 0.23570226039551587,
 'Windows 7': 0.23570226039551587,
 'Windows NT/2000/XP': 0.23570226039551587}

### Baseline

In [87]:
most_common_skills = matrix.sum(axis=0).nlargest(5)	
for i in most_common_skills.index:
    print(i)

Englisch
MySQL
SCRUM
GIT
HTML


In [109]:
p_at_5_baseline_list  = []
for i in range(0,len(matrix)):
    user_series = matrix.loc[i]
    user_know_skills = user_series[user_series > 0]
    user_known_skills_vs_most_common = user_know_skills.index.isin(most_common_skills.index)
    matches = user_known_skills_vs_most_common.sum()
    results =matches/5
    p_at_5_baseline_list.append(results)
 
p_at_5_baseline = statistics.mean(p_at_5_baseline_list)
p_at_5_baseline


0.6891089108910892

Wie aussagekräftig ist p at k ? Ein recommender, der immmer die häufigsten Skills vorschlägt ist nicht zielführend, hat haben einen hohen p_at_5 score

### Notiz
* COnfidence durch Multiplikation erhöhen
* Beste model testen und anhand eines users ausm test set nachvollziehen

The model was fitted on the whole dataset.
Try to do a train test / cross valdiation later on.
* see: https://gist.github.com/jbochi/2e8ddcc5939e70e5368326aa034a144e
* evaluation of the implicit lib: https://github.com/benfred/implicit/blob/main/implicit/evaluation.pyx